In [ ]:
import pandas as pd
import os
import numpy as np
import math
from matplotlib import pyplot as plt
from rdkit import Chem
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from utils import *
from plotting import *

In [ ]:
matchms_scores = pd.read_csv("../data/output_matching/matchms/matchms_tol_0.0035_1%I_all_peaks_with_0s_only_matching.tsv", sep="\t")
matchms_scores.rename(columns={'CosineHungarian_0.0035_0.0_1.0_scores': 'scores'}, inplace=True)
matchms_scores.rename(columns={'CosineHungarian_0.0035_0.0_1.0_matches': 'matches'}, inplace=True)

matchms_scores = append_classes(matchms_scores, 'query')
df = normalize_df(append_spectrum_metadata(matchms_scores), matches_norm_col=None)
del df['peak_comments']

matches_col = 'matches'
scores_col = 'scores'

df['matches_norm_query'] = df[matches_col] / df['n_peaks_query']
df['matches_norm_reference'] = df[matches_col] / df['n_peaks_reference']

In [ ]:
plt.scatter(df['molecular_flexibility'], df[scores_col])
plt.xlabel('Molecular Complexity')
plt.ylabel('Cosine Score')
plt.title('Cosine Score vs Molecular Complexity')
plt.show()

In [ ]:
plt.hist2d(df[scores_col], df['molecular_flexibility'], bins=10, cmap='plasma')
plt.colorbar(label='')
plt.xlabel('Scores')
plt.ylabel('Molecular Flexibility')
plt.title('2D Histogram of Scores and Molecular Flexibility')
plt.show()

In [ ]:
plt.hist2d(df[scores_col], df['molecular_complexity'], bins=10, cmap='plasma')
plt.colorbar(label='Counts')
plt.xlabel('Scores')
plt.ylabel('Molecular Complexity')
plt.title('2D Histogram of Scores and Molecular Complexity')
plt.show()

In [ ]:
plt.hist2d(df[scores_col], df['rotatable_bonds'], bins=22, cmap='plasma')
plt.colorbar(label='Counts')
plt.xlabel('Scores')
plt.ylabel('Rotatable Bonds')
plt.title('2D Histogram of Scores and Rotatable Bonds Count')
plt.show()

In [ ]:
plt.hist2d(df[scores_col], df['non_ch_atoms'], bins=10, cmap='plasma')
plt.colorbar(label='Counts')
plt.xlabel('Scores')
plt.ylabel('Non-Carbon or Hydrogen Atoms')
plt.title('2D Histogram of Scores and Atom Counts')
plt.show()

In [ ]:
plt.hist2d(df[scores_col], df['n_atoms'], bins=20, cmap='plasma')
plt.colorbar(label='Counts')
plt.xlabel('Scores')
plt.ylabel('Atoms')
plt.title('2D Histogram of Scores and Atom Counts')
plt.show()

In [ ]:
import scipy.stats as stats

stats.pearsonr(df['molecular_flexibility'].astype(float), df[scores_col].astype(float))

In [ ]:
properties = [
    'scores',
    'matches',
    'matches_norm_query',
    'matches_norm_reference',
    'molecular_flexibility',
    'rotatable_bonds',
    'stereo_centers',
    'molecular_complexity',
    'n_atoms',
    'precursor_mz',
    'electronegative_atoms',
    'aromatic_nitrogens',
    'amines',
    'amides',
]

# Assuming `df` is your DataFrame
corr = df[properties].corr().round(2)

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
# plt.title('Pearson Correlations')
plt.show()

In [ ]:
means = df[properties + ["class"]].groupby('class').mean()
medians = df[properties + ["class"]].groupby('class').median()
medians_superclass = df[properties + ["superclass"]].groupby('superclass').median()
p_means = df[properties + ["P"]].groupby('P').mean()

In [ ]:
medians.to_csv("class_medians.csv")
means.to_csv("class_means.csv")

In [ ]:
wang_db = pd.read_csv("../data/reference/13321_2020_470_MOESM1_ESM.csv")
wang_props = ["RBN", "Dot", "Mass", "Match", "number_of_atoms", "Cos", "R.match", "RBF", "PHI"]
# Assuming `df` is your DataFrame
corr_wang = wang_db[wang_props].corr().round(2)

plt.figure(figsize=(10, 8))
sns.heatmap(corr_wang, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
# plt.title('Pearson Correlations')
plt.show()

In [ ]:
wang_colwise_means = wang_db[wang_props].mean()
our_means = df[properties].mean()